In [3]:
# 预测泰坦尼克号的生存率
# 数据集 https://www.kaggle.com/c/titanic/data
# 读取数据
import pandas as pd
df = pd.read_csv('./6.titanic_train.csv')
df.info()

/home/haormj/Develop/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


/home/haormj/Develop/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [4]:
# 其中Age/Cabin/Embarked都有缺失数据,需要对数据进行预处理,现在我可以对整个数据先进行预处理
# 后续预测数据的时,也需要执行相同的预处理,从而在划分训练集和测试集后进行预处理也是可以的
# 这样的话,将同样的预处理手段应用到测试集,这样就可以后续预测时的操作是完全一致的
# 对于非数值的数据需要进行处理,离散的值可以使用one-hot编码,尤其是类别否则基于对他们的连续值进行训练没有任何意义
# 值的大小并不能够给代表什么,会给后续的预测带来问题,连续的数值则不用处理
# 还需要对缺失的值进行补充,或者根据自己的分析对某些特征直接去掉

for c in df.columns:
    print('#', c)

# PassengerId
# Survived
# Pclass
# Name
# Sex
# Age
# SibSp
# Parch
# Ticket
# Fare
# Cabin
# Embarked


In [5]:
# PassengerId 乘客ID
# Survived 是否存活,目标,最终输出值
# Pclass 船舱等级
# Name 名称
# Sex 性别
# Age 年龄
# SibSp 兄妹
# Parch 子女
# Ticket 票
# Fare 费用
# Cabin 舱号
# Embarked 上船地

In [6]:
# 乘客编号感觉不会影响到是否存活
df['PassengerId'][:5]

0    1
1    2
2    3
3    4
4    5
Name: PassengerId, dtype: int64

In [7]:
# 理论上就像飞机一样,头等舱和普通舱安全系数都是相同的,只是舒适程度不同
# 从而个人感觉应该无关,不过不知道船上是否有头等舱专用逃生艇
df['Pclass'][:5]

0    3
1    1
2    3
3    1
4    3
Name: Pclass, dtype: int64

In [8]:
# 名称我个人感觉也不会影响
df['Name'][:5]

0                              Braund, Mr. Owen Harris
1    Cumings, Mrs. John Bradley (Florence Briggs Th...
2                               Heikkinen, Miss. Laina
3         Futrelle, Mrs. Jacques Heath (Lily May Peel)
4                             Allen, Mr. William Henry
Name: Name, dtype: object

In [9]:
# 性别绝对会影响,比如男的在身体方面会比女的力量强一些,按照常理来说
df['Sex'][:5]

0      male
1    female
2    female
3    female
4      male
Name: Sex, dtype: object

In [10]:
# 年龄也会有影响
df['Age'][:5]

0    22.0
1    38.0
2    26.0
3    35.0
4    35.0
Name: Age, dtype: float64

In [11]:
# 互相帮助
df['SibSp'][:5]

0    1
1    1
2    0
3    1
4    0
Name: SibSp, dtype: int64

In [12]:
# 互相帮助
df['Parch'][:5]

0    0
1    0
2    0
3    0
4    0
Name: Parch, dtype: int64

In [13]:
# 感觉没用
df['Ticket'][:5]

0           A/5 21171
1            PC 17599
2    STON/O2. 3101282
3              113803
4              373450
Name: Ticket, dtype: object

In [14]:
# 消费一定程度上代表了家庭的财产情况,对于经济情况好的,可能会收到良好的教育
# 其中可能就包含了求生,从而我感觉应该留着
df['Fare'][:5]

0     7.2500
1    71.2833
2     7.9250
3    53.1000
4     8.0500
Name: Fare, dtype: float64

In [15]:
# 这个我持保留态度,感觉通过这个是否可以确认所在的位置,而发生事故时,和所在位置应该有很大的关系
df['Cabin'][:5]

0     NaN
1     C85
2     NaN
3    C123
4     NaN
Name: Cabin, dtype: object

In [16]:
# 上传地持保留态度
df['Embarked'][:5]

0    S
1    C
2    S
3    S
4    S
Name: Embarked, dtype: object

In [17]:
df['Survived'][:5]

0    0
1    1
2    1
3    1
4    0
Name: Survived, dtype: int64

In [75]:
# 特征 Sex/Age/SibSp/Parch/Fare
# 目标值 Survived
# 其中Age可以用平均值填充,Sex可以使用one-hot编码
# 目标值符合规范不需要处理
from sklearn.preprocessing import LabelEncoder
# sex
sex_le = LabelEncoder()
sex_le_data = sex_le.fit_transform(df['Sex'].values)

# age
# 计算平均值是否正确?
import math
age = df['Age']
age_mean = age.mean()
for i, a in enumerate(age):
    if math.isnan(a):
        age[i] = age_mean

import numpy as np
X = np.vstack((sex_le_data, age.values,df['SibSp'].values, df['Parch'].values, df['Fare'].values))
X = X.transpose()
# 最终需要对Sex那列进行OneHotEncoder
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(categorical_features=[0])
X = ohe.fit_transform(X).toarray()
y = df['Survived'].values

X.shape, y.shape

/home/haormj/Develop/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


((891, 6), (891,))

In [89]:
# 因为是分类问题,从而可以使用分类算法
# 逻辑回归,支持向量机,决策树,随机森林
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y)

from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier()
rfc.fit(X_train, y_train)
print('RandomForestClassifier', rfc.score(X_train, y_train), ' ', rfc.score(X_test, y_test))

from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(X_train, y_train)
print('LogisticRegression', lr.score(X_train, y_train), ' ', lr.score(X_test, y_test))

from sklearn.neighbors import KNeighborsClassifier
knc = KNeighborsClassifier()
knc.fit(X_train, y_train)
print('KNeighborsClassifier', knc.score(X_train, y_train), ' ', knc.score(X_test, y_test))

RandomForestClassifier 0.9662921348314607   0.8171641791044776
LogisticRegression 0.7800963081861958   0.8208955223880597
KNeighborsClassifier 0.7961476725521669   0.7313432835820896


/home/haormj/Develop/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/haormj/Develop/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [ ]:
# 本次主要是尝试xgboost算法, 看在默认参数下的准确率